## Part 1: Preprocessing

In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
columns = ['Education',
                'Age',
                'DistanceFromHome',
                'JobSatisfaction',
                'OverTime',
                'StockOptionLevel',
                'WorkLifeBalance',
                'YearsAtCompany',
                'YearsSinceLastPromotion',
                'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_df,y_df, random_state=1)


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)
X_train['OverTime'].value_counts()

,count
OverTime,
0,798
1,304


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# Create a OneHotEncoder for the Department column
department_ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
department_ohe.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = department_ohe.transform(y_train[['Department']])
y_test_department = department_ohe.transform(y_test[['Department']])

y_train_department



array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
attrition_ohe.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_ohe.transform(y_train[['Attrition']])
y_test_attrition = attrition_ohe.transform(y_test[['Attrition']])

y_train_attrition


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data
input_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)


In [14]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu',
                                       name='department_hidden')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department.shape[1],
                                       activation='softmax',
                                       name='department_output')(department_hidden_layer)


In [15]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu',
                                      name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition.shape[1],
                                      activation='sigmoid',
                                      name='attrition_output')(attrition_hidden_layer)


In [16]:
# Create the model

model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_department,
                     'attrition_output': y_train_attrition},
                    epochs=100,
                    batch_size=32,
                    verbose=1)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - attrition_output_accuracy: 0.7571 - department_output_accuracy: 0.5310 - loss: 1.5990
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8545 - department_output_accuracy: 0.6717 - loss: 1.2097
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8595 - department_output_accuracy: 0.6971 - loss: 1.0984
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8613 - department_output_accuracy: 0.6639 - loss: 1.1185
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8500 - department_output_accuracy: 0.6609 - loss: 1.1078
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8441 - department_output_accuracy: 0.6359 - loss: 1.1503
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8615 - department_output_accuracy: 0.6604 - loss: 1.1024
Epoch 8/100
35/35 ━━━━━━━━━

In [18]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled,
                              {'department_output': y_test_department,
                               'attrition_output': y_test_attrition},
                              verbose=1)
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.7877 - department_output_accuracy: 0.5335 - loss: 3.7520  


[3.858320474624634, 0.7989130616188049, 0.5380434989929199]

In [19]:
# Print the accuracy for both department and attrition
print(f"Department predicions accuracy: {test_results[2]}")
print(f"Attrition predictions accuracy: {test_results[1]}")

Department predicions accuracy: 0.5380434989929199
Attrition predictions accuracy: 0.7989130616188049


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuaracy is not the best metric because its only giving partial information, its only accounting for the true negatives and true positives. If you also want target reducing the false negatives and false positives, we have to look into metrics like precision and recall or f1 score (a combination of both)
2. For the attrition output layer I used sigmoid and for departement I used softmax. The reason I used sofmax for department is because softmax works well with multiclass outputs compared to sigmoid.
3. In the pre processing step:
- I can probably look at the correlation of imput features with labels and filter out some of the least correlated features to see if that helps improving the model.
- I can try using feature importnaces from random forest model to rank the features and exclude few of the features that havce the lowest ranks and see if that improves the model.

In the modeling step:
- I can modify the number of nodes in the existing hidden layers (both shared and individual layers).
- I can also try increasing the number of shared layers
- Try out other activation functions
- Playing around with the number of epochs
